In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.csv as csv
import datetime
import folium
import time
import geopandas

In [2]:
data=pd.read_csv('../../../data/stop_activity_granular_2020-09-01_2020-10-31.csv', engine='pyarrow')

In [3]:
df=data.copy()

In [4]:
len(df)

22506627

In [5]:
df.head()

,OPERATION_DATE,BOOKING_ID,SCHED_DAY_TYPE_CODED_NUM,SERVICE_RTE_LIST,TRIP_COMPASS_DIR_CD,KEY_BLOCK_NUM,STOP_SEQUENCE_NUM,STOP_ID,STOP_NM,SCHED_ARRIVAL_SECS_AFTER_MIDNT,FULL_DATE,ACTUAL_ARRIVAL_SECS_AFTER_MIDNT,ACTUAL_DEPARTURE_SECS_AFTER_MIDNT,PSNGR_BOARDINGS,PSNGR_ALIGHTINGS,TRIP_ID,DEP_PSNGR_LOAD
0,2020-09-01,SUM20,0,65,North,6503,13.0,10020,NE 55TH ST & 37TH AVE NE,NaN,09/01/2020,110709,110709,0,0,46931260,2
1,2020-09-01,SUM20,0,120,North,12011,32.0,21560,DELRIDGE WAY SW & SW HUDSON ST,NaN,09/01/2020,123209,123209,0,0,47583182,0
2,2020-09-01,SUM20,0,107,NULL,10714,0.0,41890,S LANDER ST & 15TH AVE S,155200.0,09/01/2020,155345,155345,0,0,47801614,0
3,2020-09-01,SUM20,0,120,NULL,12024,1.0,98141,2ND AVE & LENORA ST,124400.0,09/01/2020,124003,124003,0,0,47583379,0
4,2020-09-01,SUM20,0,671,South,67111,5.0,61100,INTERNATIONAL BLVD & S 188TH ST,NaN,09/01/2020,133751,133818,4,0,47799588,5


In [6]:
len(set(df['FULL_DATE']))

60

In [7]:
len(set(df['STOP_ID']))

5993

In [8]:
len(set(df['SERVICE_RTE_LIST']))

152

In [9]:
len(set(df['TRIP_ID']))

82027

In [10]:
df['PSNGR_ALIGHTINGS'].describe()

count    2.250663e+07
mean     2.784130e-01
std      9.474394e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      2.080000e+02
Name: PSNGR_ALIGHTINGS, dtype: float64

In [11]:
df.isnull().sum()

OPERATION_DATE                              0
BOOKING_ID                                  0
SCHED_DAY_TYPE_CODED_NUM                    0
SERVICE_RTE_LIST                            0
TRIP_COMPASS_DIR_CD                         0
KEY_BLOCK_NUM                               0
STOP_SEQUENCE_NUM                        2441
STOP_ID                                     0
STOP_NM                                     0
SCHED_ARRIVAL_SECS_AFTER_MIDNT       17238389
FULL_DATE                                   0
ACTUAL_ARRIVAL_SECS_AFTER_MIDNT             0
ACTUAL_DEPARTURE_SECS_AFTER_MIDNT           0
PSNGR_BOARDINGS                             0
PSNGR_ALIGHTINGS                            0
TRIP_ID                                     0
DEP_PSNGR_LOAD                              0
dtype: int64

In [12]:
m_start=datetime.time(6,30,0)
m_end=datetime.time(9,0,0)
e_start=datetime.time(17,0,0)
e_end=datetime.time(19,0,0)

In [13]:
df['ACTUAL_DEPARTURE_SECS_AFTER_MIDNT']=pd.to_datetime(df['ACTUAL_DEPARTURE_SECS_AFTER_MIDNT'],unit='s')
df['ACTUAL_DEPARTURE_SECS_AFTER_MIDNT']=df['ACTUAL_DEPARTURE_SECS_AFTER_MIDNT'].dt.time

In [14]:
df_m_rush=df[(df['ACTUAL_DEPARTURE_SECS_AFTER_MIDNT']>m_start) & (df['ACTUAL_DEPARTURE_SECS_AFTER_MIDNT']<m_end)]
df_m_rush

,OPERATION_DATE,BOOKING_ID,SCHED_DAY_TYPE_CODED_NUM,SERVICE_RTE_LIST,TRIP_COMPASS_DIR_CD,KEY_BLOCK_NUM,STOP_SEQUENCE_NUM,STOP_ID,STOP_NM,SCHED_ARRIVAL_SECS_AFTER_MIDNT,FULL_DATE,ACTUAL_ARRIVAL_SECS_AFTER_MIDNT,ACTUAL_DEPARTURE_SECS_AFTER_MIDNT,PSNGR_BOARDINGS,PSNGR_ALIGHTINGS,TRIP_ID,DEP_PSNGR_LOAD
0,2020-09-01,SUM20,0,65,North,6503,13.0,10020,NE 55TH ST & 37TH AVE NE,NaN,09/01/2020,110709,06:45:09,0,0,46931260,2
23,2020-09-01,SUM20,0,156,West,15605,2.0,79960,53RD AVE S & KLICKITAT DR,NaN,09/01/2020,112901,07:21:41,0,0,47800135,1
32,2020-09-01,SUM20,0,73,South,7303,17.0,38022,UNIVERSITY WAY NE & NE RAVENNA BLVD,NaN,09/01/2020,113059,07:24:19,0,0,42019428,0
42,2020-09-01,SUM20,0,65,South,6515,18.0,24930,35TH AVE NE & 34TH AVE NE,NaN,09/01/2020,113331,07:28:51,0,0,44052451,0
51,2020-09-01,SUM20,0,45,West,4507,4.0,9561,NE PACIFIC ST & UNIVERSITY WAY NE,NaN,09/01/2020,113831,07:37:11,0,0,47377561,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22506603,2020-10-31,FAL20,1,50,East,5006,32.0,99253,SODO BUSWAY & S SPOKANE ST,NaN,10/31/2020,200948,07:49:08,0,0,47804738,2
22506606,2020-10-31,FAL20,1,50,East,5006,31.0,99263,SODO BUSWAY & S LANDER ST,200900.0,10/31/2020,200752,07:46:48,0,0,47804738,2
22506619,2020-10-31,FAL20,1,183,NULL,18302,0.0,99804,FEDERAL WAY TC,114200.0,10/31/2020,114007,07:40:07,0,0,49465592,0
22506620,2020-10-31,FAL20,1,554,NULL,55402,0.0,99811,HIGHLANDS DR NE,201400.0,10/31/2020,200929,07:48:49,0,0,41378534,0


In [16]:
# covert 'OPERATION_DATE' to datetime and initialize a reinstatement date
df['OPERATION_DATE']=pd.to_datetime(df['OPERATION_DATE'])
reinstatement_date = pd.to_datetime("2020-10-01").date()

# filter into 2 datasets: before and after
before_df = df_m_rush.loc[(df_m_rush['OPERATION_DATE'] <= reinstatement_date)]
after_df = df_m_rush.loc[(df_m_rush['OPERATION_DATE'] > reinstatement_date)]

In [17]:
def load_geoid_stops_matchings():
    """Load GEOID stops and matchings."""
    df=pd.read_csv('../../../data/bus_stop_geoid_matching.csv', engine='pyarrow')
    df['GEOID'] = df['GEOID'].astype('Int64').astype('str')
    df['STOP_ID']=df['stop_id']
    df=df[['STOP_ID', 'GEOID']]
    return df

df_stop_geoid_matchings = load_geoid_stops_matchings()
df_stop_geoid_matchings

,STOP_ID,GEOID
0,3169,53033024902
1,3170,53033023300
2,3171,53033023300
3,3172,53033023300
4,3173,53033030314
...,...,...
14509,28959,53033003100
14510,28960,53033003100
14511,28961,53033003100
14512,28962,53033003100


In [18]:
def add_GEOID(df):
    """
    Add GEOID to any dataframe, matching on 'variable STOP_ID'
    """
    ret_df=df.merge(df_stop_geoid_matchings, on='STOP_ID')
    return ret_df

def agg_by_GEOID(df):
    """
    Aggregate bus stops by GEOID. Adds GEOID using add_GEOID(), and aggregates by those GEOID codes
    
    df must have 'STOP_ID' and 'PSNGR_BOARDINGS'
    """
    df_ret = df.groupby('STOP_ID').agg({'PSNGR_BOARDINGS':'mean','STOP_NM':'first'}).reset_index()
    df_ret = add_GEOID(df_ret)
    df_ret.groupby('GEOID').agg({'PSNGR_BOARDINGS':'mean','STOP_NM':'first'}).reset_index()
    return df_ret

In [19]:
before_df_agg = agg_by_GEOID(before_df)
after_df_agg = agg_by_GEOID(after_df)

In [20]:
print(f'The two bus stops that have data before but not after: {set(before_df_agg.STOP_ID) - set(after_df_agg.STOP_ID)}')

The two bus stops that have data before but not after: {17280, 10654}


In [21]:
before_df_agg.describe()

,STOP_ID,PSNGR_BOARDINGS
count,788.000000,788.000000
mean,12049.513959,0.259261
std,6316.519031,0.413479
min,3190.000000,0.000000
25%,7315.000000,0.036260
50%,10651.500000,0.108454
75%,16102.250000,0.319182
max,28680.000000,4.348101


In [22]:
after_df_agg.describe()

,STOP_ID,PSNGR_BOARDINGS
count,786.000000,786.000000
mean,12044.634860,0.242260
std,6321.606526,0.395848
min,3190.000000,0.000000
25%,7245.000000,0.026421
50%,10645.000000,0.097864
75%,16101.500000,0.295915
max,28680.000000,4.261438


In [23]:
0.259261 - 0.242260

0.017001000000000016

We decided to look at the mean number of passenger boardings during morning rush hours of 6:30-9 AM. 

We look at the number of passenger boardings in the morning because we think that morning boardings is a good indicator for people who work in those regions leaving for work in the morning. 

We found that the mean number of passenger boardings during morning rush hours decreased after fare reinstatement. In the month of September 2020, there was an average of 0.259 boardings per stop in each census tract. In the month of October 2020, after the reinstatement date of October 1st, there was an average of 0.242 boardings per stop in each census tract.

This is for boardings during morning hours specifically - I think looking at boardings during rush hours specifically will be useful for when we look at location-specific data, because I’m assuming that boardings in morning rush hours usually indicates that folks live in the census tract at those bus stops.

One thing I think we will have to investigate or add to our analysis was the fact that routes were changed during this time - perhaps we can look at the cumulative number of boardings at each bus stop as opposed to average to account for the route changes just to see the # of unique individuals using the bus

In [ ]:

from scipy.stats import ttest_ind

def get_treatment_table(group):
    # filter for those GEOID's
    before_df_agg1 = before_df_agg[before_df_agg['GEOID'].isin(group)]
    after_df_agg1 = after_df_agg[after_df_agg['GEOID'].isin(group)]

    # put them into one table: 
    ret = pd.concat([before_df_agg1.describe().PSNGR_BOARDINGS, after_df_agg1.describe().PSNGR_BOARDINGS], axis=1)

    # rename column names: 
    ret.columns.values[0] = "Before"
    ret.columns.values[1] = "After"

    return ret

def get_t_test(group):
    before_group = before_df_agg[before_df_agg['GEOID'].isin(group)]
    after_group = after_df_agg[after_df_agg['GEOID'].isin(group)]

    statistic, pvalue = tuple(ttest_ind(before_group['PSNGR_BOARDINGS'], after_group['PSNGR_BOARDINGS']))
    
    return f'The T-statistic for  is {round(statistic, 4)} with a p-value of {round(pvalue, 4)}.'


In [103]:
# Group 0: Total group
group0 = list(set(before_df_agg.GEOID))
group0

get_treatment_table(group0)


,Before,After
count,788.000000,786.000000
mean,0.259261,0.242260
std,0.413479,0.395848
min,0.000000,0.000000
25%,0.036260,0.026421
50%,0.108454,0.097864
75%,0.319182,0.295915
max,4.348101,4.261438


In [108]:
get_t_test(group0)

'The T-statistic is 0.8332 with a p-value of 0.4049.'

In [91]:
# Group 1: Black and Poor
group1 = [53033010701,53033011001,53033010001,53033011002,53033000700,53033011200,53033020700,53033020900,53033021100,53033026600,53033026801,53033027100,53033026500,53033027200,53033028000,53033028200,53033028402,53033029504,53033030501,53033030305,53033028500,53033029004,53033029203,53033026400,53033031908,53033010702,53033030006,53033030314,53033030313,53033025302,53033009200,53033009400,53033029206,53033026200,53033008500,53033008700,53033009000,53033011101]
group1 = [str(x) for x in group1]

get_treatment_table(group1)



,Before,After
count,101.000000,101.000000
mean,0.233614,0.198361
std,0.305240,0.276378
min,0.000000,0.000000
25%,0.045226,0.029703
50%,0.107884,0.094675
75%,0.331288,0.267327
max,2.032653,1.853755


In [92]:
get_t_test(group1)

'The T-statistic is 0.8604 with a p-value of 0.3906.'

In [93]:
# Group 2: White and Poor
group2 = [53033020200,53033021903,53033027701,53033031302,53033028600,53033031400,53033007600,53033032319,53033000402,53033010900]
group2 = [str(x) for x in group2]

get_treatment_table(group2)


,Before,After
count,24.000000,24.000000
mean,0.325712,0.325832
std,0.517863,0.560708
min,0.000000,0.000000
25%,0.029159,0.005326
50%,0.107909,0.087663
75%,0.383360,0.343209
max,2.074510,2.319703


In [97]:
get_t_test(group2)

'The T-statistic is -0.0008 with a p-value of 0.9994.'

In [72]:
# Group 3: Percent asian top 25 percentile

group3 = [53033010401,53033010701,53033011001,53033025805,53033010001,53033011002,53033000700,53033011200,53033020401,53033020700,53033020900,53033011700,53033025803,53033021100,53033026600,53033011102,53033029205,53033026801,53033027100,53033026500,53033027200,53033027300,53033028000,53033028100,53033028200,53033028402,53033029305,53033028403,53033029504,53033030501,53033030801,53033030201,53033030304,53033030305,53033030306,53033030003,53033030309,53033030312,53033028500,53033028802,53033025702,53033026001,53033028902,53033029004,53033029203,53033026400,53033031908,53033031909,53033010702,53033011401,53033025806,53033029102,53033030006,53033030314,53033030313,53033010002,53033025302,53033009200,53033009300,53033009400,53033009500,53033029206,53033029101,53033007700,53033026200,53033029403,53033028300,53033030308,53033004000,53033008500,53033008700,53033008800,53033008900,53033009000,53033010200,53033010800,53033011101]
group3 = [str(x) for x in group3]

get_treatment_table(group3)


,Before,After
count,188.000000,188.000000
mean,0.237535,0.208366
std,0.297198,0.273689
min,0.000000,0.000000
25%,0.041852,0.028090
50%,0.114207,0.088620
75%,0.335088,0.308237
max,2.032653,1.853755


In [96]:
get_t_test(group3)

'The T-statistic is 0.9899 with a p-value of 0.3229.'

In [94]:
# Group 4: people in census tracts where population of people in poverty(below 200 percent federal income) is greater than 10 % percentile

group4 = [53033011001,53033010001,53033011200,53033026500,53033030501,53033029004,53033008002,53033030006,53033030314,53033004302,53033025302,53033009200,53033029206,53033001300,53033008200,53033008300,53033009000,53033009100,53033010900]
group4 = [str(x) for x in group4]

get_treatment_table(group4)

,Before,After
count,62.000000,62.000000
mean,0.292401,0.253769
std,0.378178,0.352055
min,0.000000,0.000000
25%,0.057147,0.041342
50%,0.157521,0.133043
75%,0.333082,0.316584
max,2.032653,1.853755


In [95]:
get_t_test(group4)

'The T-statistic is 0.5887 with a p-value of 0.5571.'